In [0]:
%run ./init

In [0]:
import random
import uuid
from datetime import datetime, timedelta
from pyspark.sql import Row

# Generate 70 unique call records
num_records = 70

# Generate random datetime within the past 30 days
def random_datetime():
    return datetime.now() - timedelta(days=random.randint(0, 90), hours=random.randint(0, 23), minutes=random.randint(0, 59))

# Generate a unique call ID
def generate_call_id():
    return str(uuid.uuid4())[:8]  # Shortened UUID for readability

# Generate random agent IDs (5 agents)
agent_ids = [f"AGT{str(i).zfill(3)}" for i in range(1, 6)]

# Generate random customer names (unique)
first_names = ["John", "Sarah", "Michael", "Emily", "David", "Anna", "James", "Sophia", "Robert", "Olivia",
               "Daniel", "Emma", "Matthew", "Isabella", "Andrew", "Mia", "Joshua", "Charlotte", "William", "Amelia",
               "Alexander", "Evelyn", "Henry", "Abigail", "Samuel", "Ella", "Joseph", "Scarlett", "Benjamin", "Grace",
               "Nicholas", "Chloe", "Ethan", "Lily", "Logan", "Hannah", "Christopher", "Zoe", "Nathan", "Avery",
               "Ryan", "Madison", "Jack", "Layla", "Luke", "Nora", "Dylan", "Riley", "Caleb", "Aria"]

last_names = ["Smith", "Johnson", "Brown", "Taylor", "Anderson", "Martinez", "Hernandez", "Lopez", "Gonzalez", "Wilson",
              "Clark", "Lewis", "Young", "Walker", "Allen", "King", "Wright", "Scott", "Green", "Baker",
              "Adams", "Nelson", "Carter", "Mitchell", "Perez", "Roberts", "Hall", "Evans", "Campbell", "Miller",
              "Davis", "Garcia", "Rodriguez", "Martinez", "Lopez", "Harris", "Gomez", "Diaz", "Torres", "Flores",
              "Sanchez", "Reed", "Stewart", "Murphy", "Howard", "Brooks", "Gray", "Murray", "Ford", "Castro"]

# Ensure we have exactly num_records unique customer names
customer_names = list(set([f"{random.choice(first_names)} {random.choice(last_names)}" for _ in range(num_records * 2)]))[:num_records]

# Generate random DOB (ages between 69 and 19)
def random_dob():
    return (datetime.now() - timedelta(days=random.randint(19 * 365, 69 * 365))).date()

# Generate policy numbers
def generate_policy_number():
    return f"VG{random.randint(100000, 999999)}"

# Sentiment categories
sentiments = ["Happy", "Neutral", "Frustrated", "Angry", "Confused"]

reasons_dict = {
    "Claim status inquiry": "Provide claim status update",
    "Coverage details request": "Explain coverage details",
    "Billing and premium question": "Assist with billing",
    "Finding in-network provider": "Find in-network provider",
    "Policy renewal": "Initiate policy renewal",
    "Updating personal details": "Update customer details",
    "Technical support": "Provide technical support",
    "Filing a new claim": "File new claim request",
    "Canceling a policy": "Process policy cancellation"
}

financial_hardship_dict = {
    "Requesting premium payment deferral due to financial hardship": "Review eligibility for payment deferral",
    "Inquiry about hardship assistance programs": "Explain available financial hardship assistance options",
    "Request to lower coverage temporarily due to income loss": "Adjust policy coverage as requested"
}

fraud_dict = {
    "Fraudulent claim attempt": "Escalate suspected fraud"
}

# Combine all reasons for general selection
all_reason_mappings = list(reasons_dict.items())

# Generate call durations (up to 5 minutes max)
def random_call_duration():
    return random.randint(30, 300)  # 30 sec to 5 min

# Assign 10 fraud and 3 financial hardship randomly
fraud_indices = set(random.sample(range(num_records), 10))
remaining_indices = list(set(range(num_records)) - fraud_indices)
hardship_indices = set(random.sample(remaining_indices, 3))

# Generate the dataframe
call_data = []
for i in range(num_records):
    if i in fraud_indices:
        reason, step = list(fraud_dict.items())[0]
    elif i in hardship_indices:
        reason, step = random.choice(list(financial_hardship_dict.items()))
    else:
        reason, step = random.choice(all_reason_mappings)

    call_data.append(Row(
        datetime=random_datetime(),
        call_id=generate_call_id(),
        agent_id=random.choice(agent_ids),
        customer_name=customer_names[i],
        dob=random_dob(),
        policy_number=generate_policy_number(),
        sentiment=random.choice(sentiments),
        reason_for_call=reason,
        next_steps=step,
        duration_sec=random_call_duration()
    ))

# Convert to Spark DataFrame
df_calls = spark.createDataFrame(call_data)
display(df_calls)

datetime,call_id,agent_id,customer_name,dob,policy_number,sentiment,reason_for_call,next_steps,duration_sec
2025-02-26T10:50:38.548655Z,1d703bcd,AGT001,Aria Martinez,2004-03-26,VG145508,Angry,Policy renewal,Initiate policy renewal,41
2025-02-27T04:59:38.548741Z,8c5609bf,AGT001,Riley Diaz,1988-12-31,VG263766,Frustrated,Request to lower coverage temporarily due to income loss,Adjust policy coverage as requested,164
2025-02-22T09:10:38.54878Z,40f0da70,AGT001,Charlotte King,1996-01-11,VG531775,Happy,Finding in-network provider,Find in-network provider,263
2025-03-22T06:13:38.548813Z,841e1390,AGT003,Isabella Murray,1987-12-31,VG307713,Neutral,Policy renewal,Initiate policy renewal,52
2025-01-29T08:42:38.548844Z,d151e39d,AGT001,Ella Campbell,1972-12-15,VG381873,Confused,Technical support,Provide technical support,100
2025-02-15T01:29:38.548874Z,825a8ada,AGT004,Christopher Scott,1984-02-11,VG240942,Happy,Canceling a policy,Process policy cancellation,212
2025-02-17T04:22:38.548903Z,f3e78a44,AGT002,Jack Brown,1992-10-29,VG352361,Frustrated,Billing and premium question,Assist with billing,277
2025-02-17T08:20:38.548931Z,69d5f0e4,AGT002,Isabella Walker,2001-01-15,VG136372,Angry,Technical support,Provide technical support,299
2025-01-18T14:49:38.548959Z,a19fe548,AGT005,Emma Flores,2004-02-23,VG435704,Neutral,Billing and premium question,Assist with billing,292
2025-01-19T01:01:38.548987Z,6e786a42,AGT003,Scarlett Martinez,1967-12-08,VG297603,Confused,Policy renewal,Initiate policy renewal,262


In [0]:
from pyspark.sql.functions import when

# Combine all_reason_mappings, financial_hardship_dict, and fraud_dict
combined_reason_mappings = all_reason_mappings + list(financial_hardship_dict.items()) + list(fraud_dict.items())

# Convert combined_reason_mappings into a DataFrame
df_reasons = spark.createDataFrame(combined_reason_mappings, ["reason_for_call", "next_steps"])

# Update the rows where reason_for_call comes from financial_hardship_dict to 'Financial hardship'
df_reasons = df_reasons.withColumn(
    "reason_for_call",
    when(df_reasons["reason_for_call"].isin(list(financial_hardship_dict.keys())), "Financial hardship").otherwise(df_reasons["reason_for_call"])
)

display(df_reasons)

# Save the DataFrame as a table
df_reasons.write.format("delta").option("overwriteSchema", "true").mode("overwrite").saveAsTable(f"{CATALOG}.{SCHEMA}.call_centre_reasons")

reason_for_call,next_steps
Claim status inquiry,Provide claim status update
Coverage details request,Explain coverage details
Billing and premium question,Assist with billing
Finding in-network provider,Find in-network provider
Policy renewal,Initiate policy renewal
Updating personal details,Update customer details
Technical support,Provide technical support
Filing a new claim,File new claim request
Canceling a policy,Process policy cancellation
Financial hardship,Review eligibility for payment deferral


In [0]:
prompt_template = """
CONCAT('Context:  
You are an agent working for **VitalGuard**, a UK-based health insurance provider. You are handling a customer service call at the VitalGuard call centre. The conversation should be professional, clear, and helpful, following the standard call flow.

---

### Customer Details:  
- Customer Name: ' ,customer_name, '\n  
- Date of Birth: ' ,dob, '\n  
- Policy Number: ' ,policy_number, '\n  
- Customer Sentiment: ' ,sentiment, '\n  

---

### Call Scenario:  
- Reason for Call: ' ,reason_for_call, '\n  
- Next Steps: ' ,next_steps, '\n  
- Call Duration: ' ,duration_sec, ' seconds  

---

### Transcript Guidelines:  
Follow the structured conversation format:  
1. **Greeting & Verification:** The agent greets the customer and verifies their identity (name, DOB, policy number).  
2. **Understanding the Issue:** The agent asks clarifying questions based on the **Reason for Call**.  
3. **Providing Assistance:** The agent gives solutions, answers questions, and explains the next steps clearly.  
4. **Handling Concerns:** If the customer is **frustrated, concerned, or angry**, the agent should use a calm and reassuring tone. If the customer is **happy or relieved**, the agent should maintain an upbeat and professional tone.  
5. **Summarizing & Closing:** The agent recaps the next steps and ends the call politely.  

---

### Example Input:  
Customer Name: Emma Johnson  
Date of Birth: 07/22/1990  
Policy Number: VG434271  
Customer Sentiment: Happy  
Reason for Call: Seeking advice on coverage  
Next Steps: Provide complaint reference number  
Call Duration: 188 seconds  

---

### Example Output:  

Agent: "Hello, thank you for calling VitalGuard. My name is Agent. How can I assist you today?"  

Customer: "Hi, this is Emma Johnson. I was hoping to get some advice on my coverage options."  

Agent: "Of course, Emma. For security, can I confirm your date of birth and policy number?"  

Customer: "Sure, my DOB is 22nd July 1990, and my policy number is VG434271."  

Agent: "Thank you for confirming. What kind of coverage details are you looking for?"  

Customer: "I want to know if my policy covers physiotherapy sessions."  

Agent: "Let me check that for you... Yes, your current plan includes up to 10 physiotherapy sessions per year, but they need to be prescribed by a GP."  

Customer: "That is great to know! Also, if I need to make a complaint about a billing issue, how do I do that?"  

Agent: "I understand. I will provide you with a complaint reference number, and you can submit your complaint through our online portal or over the phone. Here is your reference: VG-CMP-4823."  

Customer: "Thanks, that helps!"  

Agent: "You are very welcome. Is there anything else I can assist you with today?"  

Customer: "No, that is all for now."  

Agent: "Great! Just to summarize, I have confirmed your policy covers physiotherapy with a GP referral, and I have provided a complaint reference number for your billing issue. If you need further assistance, don’t hesitate to reach out. Have a wonderful day!"  

Customer: "Thanks! You too."  

Agent: "Goodbye!"')  
"""


response_format = """{
    "type": "json_schema",
    "json_schema": {
        "name": "vitalguard_call_transcript",
        "schema": {
            "type": "object",
            "properties": {
                "customer_name": {
                    "type": "string",
                    "description": "The full name of the customer calling the VitalGuard support centre."
                },
                "dob": {
                    "type": "string",
                    "description": "The date of birth of the customer for verification purposes."
                },
                "policy_number": {
                    "type": "string",
                    "description": "The unique policy number associated with the customer."
                },
                "sentiment": {
                    "type": "string",
                    "enum": ["Calm", "Frustrated", "Concerned", "Neutral", "Angry", "Happy", "Confused", "Relieved"],
                    "description": "The sentiment of the customer during the call."
                },
                "reason_for_call": {
                    "type": "string",
                    "description": "The primary reason the customer is calling."
                },
                "next_steps": {
                    "type": "string",
                    "description": "A summary of the next actions that will be taken following the call."
                },
                "duration_sec": {
                    "type": "number",
                    "description": "The total duration of the call in seconds."
                },
                "transcript": {
                    "type": "array",
                    "description": "A structured call transcript representing the conversation between the agent and the customer.",
                    "items": {
                        "type": "object",
                        "properties": {
                            "speaker": {
                                "type": "string",
                                "enum": ["Agent", "Customer"],
                                "description": "Indicates whether the statement was made by the agent or the customer."
                            },
                            "message": {
                                "type": "string",
                                "description": "The specific statement made during the call."
                            }
                        },
                        "required": ["speaker", "message"]
                    }
                }
            },
            "required": [
                "customer_name",
                "dob",
                "policy_number",
                "sentiment",
                "reason_for_call",
                "next_steps",
                "duration_sec",
                "transcript"
            ]
        },
        "strict": true
    }
}"""

In [0]:
df_calls.createOrReplaceTempView("scenarios_temp")

query = f"""
    SELECT *,
          ai_query('{ENDPOINT_NAME}', {prompt_template}, responseFormat => '{response_format}') AS script
    FROM scenarios_temp
"""

df_scripts = spark.sql(query)

# display(df_scripts)

df_scripts.write.format("delta").option("overwriteSchema", "true").mode("overwrite").saveAsTable(f"{CATALOG}.{SCHEMA}.call_centre_scenarios")

datetime,call_id,agent_id,customer_name,dob,policy_number,sentiment,reason_for_call,next_steps,duration_sec,script
2025-02-26T10:50:38.548655Z,1d703bcd,AGT001,Aria Martinez,2004-03-26,VG145508,Angry,Policy renewal,Initiate policy renewal,41,"{""customer_name"": ""Aria Martinez"", ""dob"": ""2004-03-26"", ""sentiment"": ""Angry"", ""transcript"": [{""speaker"": ""Agent"", ""message"": ""Hello, thank you for calling VitalGuard. My name is Agent. How can I assist you today?""}, {""speaker"": ""Customer"", ""message"": ""Hi, this is Aria Martinez. I'm calling about my policy renewal.""}, {""speaker"": ""Agent"", ""message"": ""Of course, Aria. For security, can I confirm your date of birth and policy number?""}, {""speaker"": ""Customer"", ""message"": ""My DOB is 26th March 2004, and my policy number is VG145508.""}, {""speaker"": ""Agent"", ""message"": ""Thank you for confirming. I understand you're calling about your policy renewal. Can you tell me more about what's concerning you?""}, {""speaker"": ""Customer"", ""message"": ""I just want to know why my premium is increasing. I don't think it's fair.""}, {""speaker"": ""Agent"", ""message"": ""I apologize for the frustration, Aria. Let me look into this for you. (pause) I see that the premium increase is due to a change in your age bracket and an adjustment in our pricing structure. However, I can offer you some options to reduce your premium.""}, {""speaker"": ""Customer"", ""message"": ""What options?""}, {""speaker"": ""Agent"", ""message"": ""We could discuss adjusting your coverage level or adding a deductible to lower your premium. Would you like me to explain these options in more detail?""}, {""speaker"": ""Customer"", ""message"": ""Yes, please.""}, {""speaker"": ""Agent"", ""message"": ""Certainly. (explains options)... and we can also consider other discounts you might be eligible for. Shall I proceed with the policy renewal with the adjustments we've discussed?""}, {""speaker"": ""Customer"", ""message"": ""Yes, that sounds good.""}, {""speaker"": ""Agent"", ""message"": ""Great! Just to summarize, I have initiated your policy renewal with the agreed-upon adjustments to reduce your premium. You will receive a confirmation email with the details. If you have any further questions or concerns, please don't hesitate to reach out. Is there anything else I can assist you with today?""}, {""speaker"": ""Customer"", ""message"": ""No, that's all.""}, {""speaker"": ""Agent"", ""message"": ""You're welcome, Aria. Thank you for choosing VitalGuard. Have a good day!""}], ""reason_for_call"": ""Policy renewal"", ""duration_sec"": 41, ""policy_number"": ""VG145508"", ""next_steps"": ""Initiate policy renewal""}"
2025-02-27T04:59:38.548741Z,8c5609bf,AGT001,Riley Diaz,1988-12-31,VG263766,Frustrated,Request to lower coverage temporarily due to income loss,Adjust policy coverage as requested,164,"{""customer_name"": ""Riley Diaz"", ""dob"": ""1988-12-31"", ""sentiment"": ""Frustrated"", ""transcript"": [{""speaker"": ""Agent"", ""message"": ""Hello, thank you for calling VitalGuard. My name is Agent. How can I assist you today?""}, {""speaker"": ""Customer"", ""message"": ""Hi, this is Riley Diaz. I'm calling because I've recently experienced an income loss and I was wondering if it's possible to lower my coverage temporarily.""}, {""speaker"": ""Agent"", ""message"": ""I'm so sorry to hear that, Riley. For security, can I confirm your date of birth and policy number?""}, {""speaker"": ""Customer"", ""message"": ""My DOB is 31st December 1988, and my policy number is VG263766.""}, {""speaker"": ""Agent"", ""message"": ""Thank you for confirming, Riley. I understand that you're going through a tough time financially and you'd like to adjust your coverage. Can you tell me a bit more about what you're looking for in terms of temporary adjustments?""}, {""speaker"": ""Customer"", ""message"": ""Ideally, I'd like to reduce my monthly payments as much as possible without losing all my b

In [0]:
from pyspark.sql.functions import col, get_json_object, expr, udf
from pyspark.sql.types import StringType
import json

def extract_messages(transcript):
    return ' '.join([x['message'] for x in json.loads(transcript)])

extract_messages_udf = udf(extract_messages, StringType())

In [0]:
df = spark.table(f"{CATALOG}.{SCHEMA}.call_centre_scenarios").select("datetime", "call_id", "agent_id", "duration_sec", "script")
new_df = df.withColumn("transcript", get_json_object(col("script"), "$.transcript"))
new_df = new_df.withColumn("messages", extract_messages_udf(col("transcript"))).drop("script", "transcript")
# display(new_df)

datetime,call_id,agent_id,duration_sec,messages
2025-02-23T21:08:38.549966Z,0927147f,AGT004,262,"Hello, thank you for calling VitalGuard. My name is Agent. How can I assist you today? Hi, this is Caleb Clark. I'm calling because I'm having trouble finding an in-network provider. I apologize for the inconvenience, Caleb. For security, can I confirm your date of birth and policy number? My DOB is 20th August 1961, and my policy number is VG143048. Thank you for confirming. I understand you're looking for an in-network provider. Can you please tell me what type of specialist you're looking for and your location? I need a cardiologist in London. Let me check our network for you... Okay, I have found a few options. Would you like me to email you the list or go through it with you over the phone? Please go through it with me. Of course, Caleb. There are three cardiologists in London who are part of our network. I can give you their names, addresses, and contact information. That would be great, thank you. You're welcome. I'm going to summarize the next steps: I will provide you with the list of in-network cardiologists in London, and you can contact them directly to schedule an appointment. If you need further assistance, don't hesitate to reach out. Is there anything else I can help you with today? No, that's all. Thank you for your help. You're welcome, Caleb. It was my pleasure to assist you. Have a great day, and please don't hesitate to contact us if you need anything else."
2025-01-10T07:08:38.549993Z,45e24c9f,AGT001,166,"Hello, thank you for calling VitalGuard. My name is Agent. How can I assist you today? Hi, this is Zoe Carter. I was hoping to discuss my policy renewal. Of course, Zoe. For security, can I confirm your date of birth and policy number? Sure, my DOB is 23rd November 1966, and my policy number is VG296243. Thank you for confirming. I see that your policy is due for renewal. Can you tell me a little bit about what you're looking for in your renewed policy? I just want to make sure everything stays the same. I like my current coverage. I understand. I'm happy to assist you with that. I'll just need to go through a few details with you to ensure a smooth renewal process. That sounds great, thank you. You're welcome. Just to confirm, I will be initiating your policy renewal as discussed. If you have any questions or need further assistance, please don't hesitate to reach out. Is there anything else I can help you with today? No, that's all for now. Thank you for your help. You're welcome, Zoe. It was my pleasure to assist you. Just to summarize, I have initiated your policy renewal, and you should receive an email with the details shortly. If you need anything else, don't hesitate to contact us. Have a great day!"
2025-01-18T20:54:38.55002Z,e303794e,AGT003,278,"Hello, thank you for calling VitalGuard. My name is Agent. How can I assist you today? Hi, this is Layla Perez. I was hoping to update my personal details. Of course, Layla. For security, can I confirm your date of birth and policy number? Sure, my DOB is 21st July 1972, and my policy number is VG639066. Thank you for confirming. What kind of personal details are you looking to update? I want to update my address and phone number. I can assist you with that. Can you please provide me with your new address and phone number? My new address is 123 Main Street, and my new phone number is 01234 567890. Thank you for providing the updated information. I have now updated your details. Is there anything else I can assist you with today? No, that is all for now. Great! Just to summarize, I have updated your personal details. If you need further assistance, don’t hesitate to reach out. Have a wonderful day!"
2025-01-27T22:36:38.550046Z,4062774f,AGT003,66,"Hello, thank you for calling VitalGuard. My name is Agent. How can I assist you today? Hi, this is Nicholas Adams. I was hoping to get some advice on my policy renewal. Of course, Nicholas. For security, can I confir

In [0]:
from pyspark.sql.functions import current_timestamp, date_format, unix_timestamp, concat, lit, col

transcriptions_df = new_df.select(
    concat(
        lit('/Volumes/'), 
        lit(CATALOG), 
        lit('/'), 
        lit(SCHEMA), 
        lit('/audio_recordings/mp3_audio_recordings/'), 
        col("call_id"), 
        lit('_'), 
        col("agent_id"), 
        lit('_'), 
        date_format(col("datetime"), 'yyyy-MM-dd HH:mm:ss'), 
        lit('.mp3')
    ).alias("file_path"),
    concat(
        col("call_id"), 
        lit('_'), 
        col("agent_id"), 
        lit('_'), 
        date_format(col("datetime"), 'yyyy-MM-dd HH:mm:ss')
    ).alias("file_name")
    col("messages").alias("transcription"),
    col("duration_sec").alias("audio_duration"),
    current_timestamp().alias("modificationTime")
)

transcriptions_df = transcriptions_df.withColumn("path", concat(lit('dbfs:'), col("file_path"))).withColumn("modificationTime", unix_timestamp(col("modificationTime")))


# transcriptions_df = transcriptions_df.select("path", "modificationTime", "file_path", "transcription", "audio_duration")
transcriptions_df = transcriptions_df.select("file_path", "file_name", "transcription", "audio_duration")

# display(transcriptions_df)

transcriptions_df.write.format("delta").option("overwriteSchema", "true").mode("overwrite").saveAsTable(f"{CATALOG}.{SCHEMA}.simulated_transcriptions")

file_path,transcription,audio_duration
/Volumes/samantha_wise/ai_claims_processing/audio_recordings/mp3_audio_recordings/0927147f_AGT004_2025-02-23 21:08:38.mp3,"Hello, thank you for calling VitalGuard. My name is Agent. How can I assist you today? Hi, this is Caleb Clark. I'm calling because I'm having trouble finding an in-network provider. I apologize for the inconvenience, Caleb. For security, can I confirm your date of birth and policy number? My DOB is 20th August 1961, and my policy number is VG143048. Thank you for confirming. I understand you're looking for an in-network provider. Can you please tell me what type of specialist you're looking for and your location? I need a cardiologist in London. Let me check our network for you... Okay, I have found a few options. Would you like me to email you the list or go through it with you over the phone? Please go through it with me. Of course, Caleb. There are three cardiologists in London who are part of our network. I can give you their names, addresses, and contact information. That would be great, thank you. You're welcome. I'm going to summarize the next steps: I will provide you with the list of in-network cardiologists in London, and you can contact them directly to schedule an appointment. If you need further assistance, don't hesitate to reach out. Is there anything else I can help you with today? No, that's all. Thank you for your help. You're welcome, Caleb. It was my pleasure to assist you. Have a great day, and please don't hesitate to contact us if you need anything else.",262
/Volumes/samantha_wise/ai_claims_processing/audio_recordings/mp3_audio_recordings/45e24c9f_AGT001_2025-01-10 07:08:38.mp3,"Hello, thank you for calling VitalGuard. My name is Agent. How can I assist you today? Hi, this is Zoe Carter. I was hoping to discuss my policy renewal. Of course, Zoe. For security, can I confirm your date of birth and policy number? Sure, my DOB is 23rd November 1966, and my policy number is VG296243. Thank you for confirming. I see that your policy is due for renewal. Can you tell me a little bit about what you're looking for in your renewed policy? I just want to make sure everything stays the same. I like my current coverage. I understand. I'm happy to assist you with that. I'll just need to go through a few details with you to ensure a smooth renewal process. That sounds great, thank you. You're welcome. Just to confirm, I will be initiating your policy renewal as discussed. If you have any questions or need further assistance, please don't hesitate to reach out. Is there anything else I can help you with today? No, that's all for now. Thank you for your help. You're welcome, Zoe. It was my pleasure to assist you. Just to summarize, I have initiated your policy renewal, and you should receive an email with the details shortly. If you need anything else, don't hesitate to contact us. Have a great day!",166
/Volumes/samantha_wise/ai_claims_processing/audio_recordings/mp3_audio_recordings/e303794e_AGT003_2025-01-18 20:54:38.mp3,"Hello, thank you for calling VitalGuard. My name is Agent. How can I assist you today? Hi, this is Layla Perez. I was hoping to update my personal details. Of course, Layla. For security, can I confirm your date of birth and policy number? Sure, my DOB is 21st July 1972, and my policy number is VG639066. Thank you for confirming. What kind of personal details are you looking to update? I want to update my address and phone number. I can assist you with that. Can you please provide me with your new address and phone number? My new address is 123 Main Street, and my new phone number is 01234 567890. Thank you for providing the updated information. I have now updated your details. Is there anything else I can assist you with today? No, that is all for now. Great! Just to summarize, I have updated your personal details. If you need further assistance, don’t hesitate to reach out. Have a wonderful day!",278
/Volumes/samantha_wise/ai_claims_proce